In [1]:
import os 
import numpy as np
import pandas as pd
import rasterio
import os
import pystac
import pystac_client
import planetary_computer
import geopandas as gpd
from rich.table import Table
import rioxarray
import requests
from shapely.geometry import Polygon, shape, box
from shapely.ops import transform
from rich.table import Table
from rioxarray.merge import merge_arrays
from rasterio.plot import plotting_extent
from rasterstats import zonal_stats
from xrspatial.zonal import stats as zonal_stats
from geocube.api.core import make_geocube
import xarray
from azureml.core import Workspace, Dataset
import multiprocessing
from functools import partial

In [1]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/p18q1262/code/Users/p18q126'

In [6]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '6c748216-f035-4623-80c3-29e1c28c7a56'
resource_group = 'geo_microsoft_general'
workspace_name = 'pop_heat_flood'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='population_data')
dataset.download(target_path='./data', overwrite=True)

In [3]:
directory = os.getcwd() + '/data'                                                                               # set directory
heat_data = rasterio.open('global_mean_wbgt_30.tif', masked=True)                                               # load heat data               

In [4]:
for fileName in os.listdir(directory):
        if fileName.endswith(".tif"):
        
            f = os.path.join(directory, fileName)
        
            countryCode = fileName[0:3]                                                                             # set country name

            outFileName = directory + '/' + countryCode + '_heat_flood_extract.csv'                                 # do not overwrite files already processes
            
            if os.path.exists(outFileName):
                print(countryCode + ' - file exists - skipping')
                continue
        
            print(f"{countryCode} - Started") # STATUS CHECK

            country_pop = rioxarray.open_rasterio(f, masked=True)                                                   # open individual pop file with filepath

            country_box = country_pop.rio.bounds()                                                                  # find bounding box for country

            country_pop.name = "pop_data"                                                                           # change column header to pop_data

            country_pop_df = country_pop.squeeze().to_dataframe().reset_index()                                     # squeeze to data frame

            geometry = gpd.points_from_xy(country_pop_df.x, country_pop_df.y)                                       # set geometry

            country_pop_gdf = gpd.GeoDataFrame(country_pop_df, crs=country_pop.rio.crs, geometry=geometry)          # convert to geopandas df

            country_pop_gdf.dropna(subset=['pop_data'], how='all', inplace=True)                                    # remove pixels with no people

            country_pop_gdf.index = range(len(country_pop_gdf))
            coords = [(x,y) for x, y in zip(country_pop_gdf.x, country_pop_gdf.y)]
            country_pop_gdf['heat_value'] = [x[0] for x in heat_data.sample(coords)]                                # extract heat data

            print(f"{countryCode} - Heat Completed") # STATUS CHECK
                
            catalog = pystac_client.Client.open(
                "https://planetarycomputer.microsoft.com/api/stac/v1", 
                modifier=planetary_computer.sign_inplace,)                                                          # access flood data on PC

            AOI = shape({ "coordinates": [[
                [country_box[0],
                 country_box[1]],
                    
                  [country_box[0],
                  country_box[3]],
                    
                  [country_box[2],
                  country_box[1]],
                    
                [country_box[2],
                 country_box[3]]
                   ]],
                  "type": "Polygon"}).envelope                                                                        # use bounding box from before to get data for each country


            jrc = catalog.search(collections=["jrc-gsw"], intersects=AOI)
            items = jrc.item_collection()                                                                           # Use STACAPI to access the collection (find the number of images for each bounding box)

            n = len(items)
            doubles = dict()
                
            for i in range(0, n):
                item = items[i]
                    
                doubles[i] = rioxarray.open_rasterio(item.assets["occurrence"].href, overview_level=4).squeeze()    # use this loop to load each image

            flood_data = merge_arrays(dataarrays = list(doubles.values()), crs = "EPSG:4326")                       # merge each image that's been loaded

            flood_data = flood_data.where(flood_data != 100)
            flood_data = flood_data.where(flood_data != 255)                                                        # Remove values of 100, and values of 255)
                
            flood_data.rio.write_nodata(flood_data.rio.nodata, encoded=True, inplace=True)                          # update nodata value to show the data has been masked

            country_pop_gdf_buff = country_pop_gdf.copy()
            country_pop_gdf_buff["geometry"] = country_pop_gdf.geometry.buffer(0.008)                               # create buffered polygon for each population point
            country_pop_gdf_buff["id"] = country_pop_gdf_buff.index + 1                                             # used to extract flood mean (also add index value here to merge back)

            out_grid = make_geocube(
                        vector_data= country_pop_gdf_buff,
                        measurements=["id"],
                        like=flood_data, 
                        )                                                                                           # make geocube for zonal stats & ensure the data are on the same grid
                    
            out_grid["flood_data"] = (flood_data.dims, flood_data.values, flood_data.attrs, flood_data.encoding)    # merge pop and flood together

            grouped_flood_data = out_grid.drop("spatial_ref").groupby(out_grid.id)
            grid_mean = grouped_flood_data.mean().rename({"flood_data": "flood_value"})                             # extract mean flood within the polygon

            print(f"{countryCode} - Flood Completed") # STATUS CHECK

            zonal_stats = xarray.merge([grid_mean]).to_dataframe()                                                  # convert resulting file to dataframe

            country_final_data = country_pop_gdf_buff.merge(zonal_stats, on='id')
            country_final_data = pd.DataFrame(country_final_data.drop(columns=['geometry', 'band', 'spatial_ref'])) # merge back in to data with heat

            country_final_data["country_iso"] = countryCode                                                         # add country code as a final column for reference
                
            country_final_data.to_csv(outFileName, index = False)

            print(f"{countryCode} - Completed & Saved") # STATUS CHECK

abw - file exists - skipping
afg - file exists - skipping
ago - file exists - skipping
aia - file exists - skipping
alb - file exists - skipping
and - file exists - skipping
are - file exists - skipping
arg - file exists - skipping
arm - file exists - skipping
asm - file exists - skipping
atg - file exists - skipping
aus - Started
aus - Heat Completed


/tmp/ipykernel_14933/2917271581.py:75: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  country_pop_gdf_buff["geometry"] = country_pop_gdf.geometry.buffer(0.008)                               # create buffered polygon for each population point


KeyboardInterrupt: 